Importar librerias

In [89]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets 
import ast
import gzip
import pandas as pd
from io import StringIO
import gdown
import requests


Crear rutas de archivos

In [96]:

# URL de descarga directa del archivo JSON desde Google Drive
ruta_reviews = 'https://drive.google.com/uc?id=1_JWEl2MCLCvYKUAJH94KoYu1aQFYBivX'


Leer archivo Json reviews

In [ ]:
def lac():
    with open(ruta_reviews, "rt", encoding="utf-8") as file:
        return [ast.literal_eval(line.strip()) for line in file]

datos = lac()
df_user_reviews_orig = pd.DataFrame(datos)


Eliminar espacios en blanco

In [22]:
# Aplicar strip() a cada celda del DataFrame para eliminar espacios en blanco
df_user_reviews_orig = df_user_reviews_orig.applymap(lambda x: x.strip() if isinstance(x, str) else x)

Crear CSV original reviews. 

Asi podemos dejar de utilizar el archivo Json que es mas pesado al momento de realizar su descarga y utilizar uno mas manejable como es CSV

In [25]:
# Ruta donde deseas guardar el archivo CSV
ruta_csv = r"C:\Users\computadora\Desktop\ProyectoIndividualML.JC\ProyectoIndividualML.JC\df_reviews_orig.csv"  # Cambia esta ruta a la que desees

# Guarda el DataFrame en un archivo CSV
df_user_reviews_orig.to_csv(ruta_csv, index=False)  # El argumento 'index=False' evita que se incluya el índice en el archivo CSV

print("DataFrame guardado como archivo CSV en:", ruta_csv)

DataFrame guardado como archivo CSV en: C:\Users\computadora\Downloads\Labs Henrys PI\ProyectoIndividualML.JC\DF_CSV_FUNCIONES\df_reviews_orig.csv


Cargar el DF desde el CSV que creamos

In [27]:
archivo = (r'../ProyectoIndividualML.JC/df_reviews_orig.csv')
df_user_reviews_orig = pd.read_csv(archivo)

utilizamos el metodo info() para visualizar el tipo de datos, la catidad de registros, catidad de columnas y el  tamaño en memoria de la data.

In [28]:
df_user_reviews_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [29]:
df_user_reviews_orig.sample(3)

,user_id,user_url,reviews
3879,10101010101010109,http://steamcommunity.com/id/10101010101010109,"[{'funny': '', 'posted': 'Posted December 15, ..."
20952,76561198058892591,http://steamcommunity.com/profiles/76561198058...,"[{'funny': '', 'posted': 'Posted March 21, 201..."
22908,bringmeababytosacrifice,http://steamcommunity.com/id/bringmeababytosac...,"[{'funny': '', 'posted': 'Posted July 25, 2015..."


Podemos visualizar la data atraves de un Datframe y evidenciamos que posee:

  * 3 columnas 
  * Una contien el user_id que podriamos determinar como la columna de identificacion unica,
  
  * luego la columna user_url que contiene datos de las url suponemos del usuario ya que al finalizar coinciden con el user_id 
  
  * y por ultimo una columna de reviews que contine datos en formato json anidados.

aca vamos a detallar los valores dentro de la columna reviews

In [30]:
df_user_reviews_orig['reviews'].sample(3)

1986     [{'funny': '1 person found this review funny',...
15105    [{'funny': '', 'posted': 'Posted December 25, ...
2060     [{'funny': '1 person found this review funny',...
Name: reviews, dtype: object

Crear DF solo con la columna user_id y reviews para desnaidar los datos que se encuentras dentro de ella

In [31]:
df_user_reviews_orig[['user_id','reviews']].head(3)

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."


Procederemos a realizar un Limpieza de Datos al DF que creamos apartir de la columna reviews y user_id
ya que con esta informacion podemos crear el dataset con el que podemos armar una de las funciones requeridas

Verificacion de datos duplicados en columna user_id.

In [32]:
duplicados = df_user_reviews_orig.duplicated('user_id')
duplicados.sum()

314

Creamos un DF solo con los valores no duplicados en la columna user_id

In [33]:
df_user_reviews_orig = df_user_reviews_orig[duplicados]
df_user_reviews_orig

,user_id,user_url,reviews
456,bokkkbokkk,http://steamcommunity.com/id/bokkkbokkk,"[{'funny': '', 'posted': 'Posted September 24,..."
1182,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,"[{'funny': '', 'posted': 'Posted January 10, 2..."
1456,76561198062039159,http://steamcommunity.com/profiles/76561198062...,"[{'funny': '', 'posted': 'Posted August 24, 20..."
1477,76561198045009232,http://steamcommunity.com/profiles/76561198045...,"[{'funny': '', 'posted': 'Posted October 31, 2..."
1746,nitr0ticwolf,http://steamcommunity.com/id/nitr0ticwolf,"[{'funny': '', 'posted': 'Posted December 12, ..."
...,...,...,...
17819,76561198076474887,http://steamcommunity.com/profiles/76561198076...,"[{'funny': '', 'posted': 'Posted April 12.', '..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
18028,76561198075591109,http://steamcommunity.com/profiles/76561198075...,"[{'funny': '', 'posted': 'Posted December 26, ..."
18234,76561198092022514,http://steamcommunity.com/profiles/76561198092...,"[{'funny': '', 'posted': 'Posted July 3.', 'la..."


In [34]:
df_user_reviews_orig.info()

<class 'pandas.core.frame.DataFrame'>
Index: 314 entries, 456 to 18309
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   314 non-null    object
 1   user_url  314 non-null    object
 2   reviews   314 non-null    object
dtypes: object(3)
memory usage: 9.8+ KB


Verificra regsitros Nulos

In [35]:
df_user_reviews_orig.isnull().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

Verificar resgitros en blancos

In [36]:
df_user_reviews_orig.isna().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

Desanidar DF datos_reviews

Observar la estructura de la primera fila de los valores

In [37]:
primera_review = df_user_reviews_orig['reviews'].iloc[0]
primera_review

"[{'funny': '', 'posted': 'Posted September 24, 2015.', 'last_edited': '', 'item_id': '346110', 'helpful': '1 of 1 people (100%) found this review helpful', 'recommend': True, 'review': 'yep'}]"

Observamos que:

poseen datos en formato de Diccionario cuyas claves son:

- funny : valor
- posted : valor        *** en este valor nos vamos a enfocar ya que necesitamos la fecha para la funcion a trabajar
- last_edited : valor
- item_id : valor
- helpful : valor
- recommend : valor      *** este dato posee valores booleanos que vamos a a transformar en valores bit 1 True y 0 False
- review : valor         *** aca este valor tambie sera utilizadon en una funcion posteriormente 

In [91]:
df_user_reviews_orig = pd.read_json('../ProyectoIndividualML.JC/user_reviews.json')
df_user_reviews_orig

,user_id,user_url,reviews
0,bokkkbokkk,http://steamcommunity.com/id/bokkkbokkk,"[{'funny': '', 'posted': 'Posted September 24,..."
1,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,"[{'funny': '', 'posted': 'Posted January 10, 2..."
2,76561198062039159,http://steamcommunity.com/profiles/76561198062...,"[{'funny': '', 'posted': 'Posted August 24, 20..."
3,76561198045009232,http://steamcommunity.com/profiles/76561198045...,"[{'funny': '', 'posted': 'Posted October 31, 2..."
4,nitr0ticwolf,http://steamcommunity.com/id/nitr0ticwolf,"[{'funny': '', 'posted': 'Posted December 12, ..."
...,...,...,...
309,76561198076474887,http://steamcommunity.com/profiles/76561198076...,"[{'funny': '', 'posted': 'Posted April 12.', '..."
310,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
311,76561198075591109,http://steamcommunity.com/profiles/76561198075...,"[{'funny': '', 'posted': 'Posted December 26, ..."
312,76561198092022514,http://steamcommunity.com/profiles/76561198092...,"[{'funny': '', 'posted': 'Posted July 3.', 'la..."


In [92]:
# Reemplaza las comillas simples por comillas dobles en la columna "reviews"
df_user_reviews_orig['reviews'] = df_user_reviews_orig['reviews'].str.replace("'", '"')

Desanidar la columna reviews

In [ ]:
# Inicializa una lista vacía para almacenar los datos de la columna "reviews"
datos_reviews = []

# Recorre el DataFrame y la columna "reviews"
for index, fila in df_user_reviews_orig.iterrows():
    for review in fila["reviews"]:
        datos_reviews.append({
            "user_id": fila["user_id"],
            "user_url": fila["user_url"],
            "funny": review.get("funny", ""),  
            "posted": review.get("posted", ""),
            "last_edited": review.get("last_edited", ""),
            "item_id": review.get("item_id", ""),
            "helpful": review.get("helpful", ""),
            "recommend":review.get("recommend", ""),
            "review":review.get("review", "")
        })

# Convierte la lista de datos en un nuevo DataFrame
datos_reviews_desanidados = pd.DataFrame(datos_reviews)




Crear CSV Desanidados reviews

In [16]:
# Ruta donde deseas guardar el archivo CSV
ruta_csv = r"C:\Users\computadora\Desktop\ProyectoML.JC\ProyectoIndividualML.JC\datos_reviews_desanidados.csv"  # Cambia esta ruta a la que desees

# Guarda el DataFrame en un archivo CSV
datos_reviews_desanidados.to_csv(ruta_csv, index=False)  # El argumento 'index=False' evita que se incluya el índice en el archivo CSV

print("DataFrame guardado como archivo CSV en:", ruta_csv)

DataFrame guardado como archivo CSV en: C:\Users\computadora\Downloads\Labs Henrys PI\datos_reviews_desanidados.csv


In [133]:
datos_reviews_desanidados = pd.read_csv('../ProyectoIndividualML.JC/datos_reviews_desanidados.csv')
datos_reviews_desanidados.head(3)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted November 5, 2011.",NaN,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted July 15, 2011.",NaN,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted April 21, 2011.",NaN,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


Crear un DF solo con la columna ['posted'] para luego desanidarla

In [134]:
def_posted = pd.DataFrame(datos_reviews_desanidados[['user_id','posted']])
def_posted.head(3)

,user_id,posted
0,76561197970982479,"Posted November 5, 2011."
1,76561197970982479,"Posted July 15, 2011."
2,76561197970982479,"Posted April 21, 2011."


Limpiar DF def_posted creado con nuevo_df_reviews['posted']

In [135]:

# Eliminar los puntos de la columna 'texto' utilizando .str.replace()
def_posted['posted'] = def_posted['posted'].str.replace('.', '')
def_posted['posted'] = def_posted['posted'].str.replace(',', '')
def_posted['posted'] = def_posted['posted'].str.replace('Posted', '')



In [136]:
def_posted.head(3)

,user_id,posted
0,76561197970982479,November 5 2011
1,76561197970982479,July 15 2011
2,76561197970982479,April 21 2011


Aca necesitamos separar la columna posted a fin de obtener la fecha en el siguiente formato
YYYY-MM-DD

Desanidar el DF def_posted

In [137]:
primera_fila = def_posted['posted'].iloc[0]
primera_fila

' November 5 2011'

Se observa que debemos:

- separar los datos en Año, Mes y dia
- Transformar los meses en numeros
- y colocar dos digitos a los dias.

In [138]:
# Dividir la columna 'posted' en nuevas columnas de día, mes y año
def_posted[['month', 'day', 'year']] = def_posted['posted'].str.split(' ', n=2, expand=True)

# Eliminar la columna 'posted' original
def_posted.drop(columns=['posted'], inplace=True)

# Eliminar la columna 'month' original
def_posted.drop(columns=['month'], inplace=True)

In [139]:
def_posted.head(3)

,user_id,day,year
0,76561197970982479,November,5 2011
1,76561197970982479,July,15 2011
2,76561197970982479,April,21 2011


In [140]:
# Usando .rename() para cambiar el nombre de la columna
def_posted = def_posted.rename(columns={'day': 'month'})

In [141]:
def_posted.head(3)

,user_id,month,year
0,76561197970982479,November,5 2011
1,76561197970982479,July,15 2011
2,76561197970982479,April,21 2011


Modificar la columna 'month' de letras a numeros str

In [142]:

# Diccionario de correspondencia entre meses y números
meses_a_numeros = {
    'January': '01',
    'February': '02',
    'March': '03',
    'April': '04',
    'May': '05',
    'June': '06',
    'July': '07',
    'August': '08',
    'September': '09',
    'October': '10',
    'November': '11',
    'December': '12'
}

# Usar .map() para transformar los nombres de los meses en números de dos dígitos
def_posted['meses_numeros'] = def_posted['month'].map(meses_a_numeros)

# Eliminar la columna 'month' original
def_posted.drop(columns=['month'], inplace=True)

In [143]:
def_posted.head(3)

,user_id,year,meses_numeros
0,76561197970982479,5 2011,11
1,76561197970982479,15 2011,07
2,76561197970982479,21 2011,04


Desagrupar la columna 'year' ya que contiene los valores de año y dia juntos

In [144]:
# Dividir la columna 'year' en nuevas columnas de día y año
def_posted[['day', 'year2']] = def_posted['year'].str.split(' ', n=2, expand=True)

# Eliminar la columna 'year' original
def_posted.drop(columns=['year'], inplace=True)


In [145]:
# cambiar el nombre de las columna
def_posted = def_posted.rename(columns={'year2': 'year'})
def_posted = def_posted.rename(columns={'meses_numeros': 'mes'})

In [146]:
# Mostrar el resultado
def_posted.sample(frac=1)

,user_id,mes,day,year
30926,76561198057784617,07,26,2013
41043,76561197971402201,02,26,2014
10450,76561198079314761,01,23,None
36146,odkul,06,4,None
43763,aetheling,08,16,2013
...,...,...,...,...
32889,76561198031056726,07,4,2014
8864,76561198071052616,09,5,2015
7578,samuelx5,12,31,2014
48567,76561198056931853,06,19,2014


verificra Nulos en el DF creado

In [147]:
def_posted.isnull().sum()

user_id       0
mes           0
day           0
year       9932
dtype: int64

Imputar valores Nulos con la la moda en la columna year

In [148]:
# ver el valor que mas se repite
def_posted["year"].mode()

# imputar el valor en la columna year
def_posted["year"].fillna(def_posted["year"].mode()[0], inplace=True)

In [149]:
def_posted.isnull().sum()

user_id    0
mes        0
day        0
year       0
dtype: int64

In [150]:
# Mostrar el resultado
def_posted.sample(frac=1)

,user_id,mes,day,year
2857,76561198095214263,01,18,2014
56312,gw4545,01,3,2014
24007,AkkarinDarkmoor,09,13,2014
44478,serpentine19,04,10,2014
41390,76561198084686700,08,29,2015
...,...,...,...,...
47797,ricardojr190,08,30,2014
11228,MAKLO_GEMING_xDDDD,10,27,2015
7406,Aussiefella_69,09,26,2015
5463,ethanabey,07,2,2014


Crear una columna completa con los datos de las columnas def_posted'['mes','day','year']. Asegurando 2 digitos para dia y mes

In [151]:
def_posted['fecha'] = def_posted.apply(lambda row: f"{row['year']}-{str(row['mes']).zfill(2)}-{str(row['day']).zfill(2)}", axis=1)

In [152]:
def_posted

,user_id,mes,day,year,fecha
0,76561197970982479,11,5,2011,2011-11-05
1,76561197970982479,07,15,2011,2011-07-15
2,76561197970982479,04,21,2011,2011-04-21
3,js41637,06,24,2014,2014-06-24
4,js41637,09,8,2013,2013-09-08
...,...,...,...,...,...
58425,76561198312638244,07,10,2014,2014-07-10
58426,76561198312638244,07,8,2014,2014-07-08
58427,LydiaMorley,07,3,2014,2014-07-03
58428,LydiaMorley,07,20,2014,2014-07-20


Cambiar de la columna ['recommend'] los valores True por 1 y los False por 0

In [153]:
datos_reviews_desanidados[['recommend']] = datos_reviews_desanidados[['recommend']].replace({True: 1, False: 0})

# Imprimir el nuevo DataFrame df_recommended
datos_reviews_desanidados[['recommend']].sample(frac=1)

,recommend
54702,1
35429,1
32757,1
3242,1
21539,1
...,...
42366,1
50891,0
6572,1
18195,1


Verificar valores nulos

In [154]:
datos_reviews_desanidados[['recommend']].isnull().sum()

recommend    0
dtype: int64

Crear el DF para la API con las columnas : ['fecha'],['user_id'],['recommend']. para funcion 2 solicitada

In [155]:
columnas_seleccionadas = [def_posted['fecha'],datos_reviews_desanidados['user_id'],datos_reviews_desanidados['recommend'] ] 
# Lista de nombres de columnas a seleccionar

# Crear un nuevo DataFrame df_original con las columnas seleccionadas
def_reviews_api2 = pd.concat(columnas_seleccionadas, axis=1)

In [156]:
def_reviews_api2.sample(frac=1)

,fecha,user_id,recommend
41379,2013-07-19,HuskyWaffle,1
17838,2015-10-20,shadowpulse29,0
31587,2013-12-26,76561198076701454,1
57102,2014-08-04,76561198106878606,1
31957,2012-07-12,djreap3r,1
...,...,...,...
40159,2014-01-05,TSGStosh,1
40283,2014-06-26,76561197997129113,1
47931,2014-11-02,green290,1
48370,2014-11-11,zeejackass,1


eliminar los valores duplicados del DF

In [157]:

# Crear un nuevo DataFrame con los valores únicos de la columna "user_id"
def_reviews_api2['user_id'] = def_reviews_api2[['user_id']].drop_duplicates()


In [163]:
def_reviews_api2 = def_reviews_api2.dropna()

In [164]:
def_reviews_api2

,fecha,user_id,recommend
0,2011-11-05,76561197970982479,1
3,2014-06-24,js41637,1
6,2014-02-03,evcentric,1
12,2013-10-14,doctr,1
18,2014-04-15,maplemage,1
...,...,...,...
58420,2014-05-31,76561198306599751,1
58421,2014-06-17,Ghoustik,1
58422,2014-06-23,76561198310819422,1
58423,2014-07-21,76561198312638244,1


Guardar el DF del API2 para la funcion 2 en un archivo CSV para usarlo posteriormente

In [165]:

# Ruta donde deseas guardar el archivo CSV
ruta_csv = r"C:\Users\computadora\Desktop\ProyectoML.JC\ProyectoIndividualML.JC\def_reviews_api2.csv"  # Cambia esta ruta a la que desees

# Guarda el DataFrame en un archivo CSV
def_reviews_api2.to_csv(ruta_csv, index=False)  # El argumento 'index=False' evita que se incluya el índice en el archivo CSV

print("DataFrame guardado como archivo CSV en:", ruta_csv) 

DataFrame guardado como archivo CSV en: C:\Users\computadora\Desktop\ProyectoML.JC\ProyectoIndividualML.JC\def_reviews_api2.csv


Llamar a DF desde el csv creado para la funcion 2 de la API

In [166]:
def_reviews_api2=pd.read_csv('../ProyectoIndividualML.JC/def_reviews_api2.csv')
def_reviews_api2

,fecha,user_id,recommend
0,2011-11-05,76561197970982479,1
1,2014-06-24,js41637,1
2,2014-02-03,evcentric,1
3,2013-10-14,doctr,1
4,2014-04-15,maplemage,1
...,...,...,...
25452,2014-05-31,76561198306599751,1
25453,2014-06-17,Ghoustik,1
25454,2014-06-23,76561198310819422,1
25455,2014-07-21,76561198312638244,1


In [167]:

# Contar las ocurrencias únicas del usuario (user_id) en la columna 'user_id'
unique_user_count = def_reviews_api2['user_id'].nunique()

# Mostrar el resultado
print(unique_user_count)

25457


def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [168]:
def_reviews_api2

,fecha,user_id,recommend
0,2011-11-05,76561197970982479,1
1,2014-06-24,js41637,1
2,2014-02-03,evcentric,1
3,2013-10-14,doctr,1
4,2014-04-15,maplemage,1
...,...,...,...
25452,2014-05-31,76561198306599751,1
25453,2014-06-17,Ghoustik,1
25454,2014-06-23,76561198310819422,1
25455,2014-07-21,76561198312638244,1


In [169]:


def countreviews(fecha_inicio, fecha_fin):
    # Filtrar el DataFrame por las fechas dadas
    df_filtrado = def_reviews_api2[(def_reviews_api2['fecha'] >= fecha_inicio) & (def_reviews_api2['fecha'] <= fecha_fin)]

    # Calcular la cantidad de usuarios únicos
    usuarios_unicos = df_filtrado['user_id'].nunique()

    # Calcular el porcentaje de recomendación promedio
    porcentaje_promedio = df_filtrado['recommend'].mean() * 100

    # Calcular la cantidad de items
    cantidad_items = len(df_filtrado)

    return usuarios_unicos, porcentaje_promedio, cantidad_items


In [170]:

# Ejemplo de uso
fecha_inicio = '2011-01-01'
fecha_fin = '2014-12-31'

usuarios, porcentaje, cantidad_items = countreviews(fecha_inicio, fecha_fin)

print(f"Cantidad de usuarios que realizaron reviews entre {fecha_inicio} y {fecha_fin}: {usuarios}")
print(f"Porcentaje de recomendación promedio: {porcentaje:.2f} %")
print(f"Cantidad de items: {cantidad_items}")

Cantidad de usuarios que realizaron reviews entre 2011-01-01 y 2014-12-31: 17542
Porcentaje de recomendación promedio: 90.45 %
Cantidad de items: 17542
